## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
wdi_df = pd.read_csv("data/WDIData.csv")
wdi_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.392349,17.892005,18.359993,18.795151,19.295176,19.788156,20.279599,20.773627,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.720331,7.015917,7.281390,7.513673,7.809566,8.075889,8.366010,8.684137,NaN,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,38.184152,38.543180,38.801719,39.039014,39.323186,39.643848,39.894830,40.213891,NaN,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.859257,33.903515,38.851444,40.197332,43.028332,44.389773,46.268621,48.103609,NaN,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.623956,16.516633,24.594474,25.389297,27.041743,29.138285,30.998687,32.772690,NaN,NaN


In [3]:
wdi_df.drop('Unnamed: 67', axis=1, inplace=True)

In [4]:
id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']

wdi_melt = pd.melt(wdi_df, 
                   id_vars=id_vars, 
                   var_name='Years', 
                   value_name='Score')
wdi_melt.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Years,Score
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,1960,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,1960,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,1960,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,1960,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,1960,NaN


In [5]:
series_df = pd.read_csv("data/WDISeries.csv")
series_df.head(2)

,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,...,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
0,AG.AGR.TRAC.NO,Environment: Agricultural production,"Agricultural machinery, tractors",NaN,Agricultural machinery refers to the number of...,NaN,Annual,NaN,NaN,Sum,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",A tractor provides the power and traction to m...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN
1,AG.CON.FERT.PT.ZS,Environment: Agricultural production,Fertilizer consumption (% of fertilizer produc...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,The world and regional aggregate series do not...,Weighted average,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0,NaN


In [6]:
eastern_countries = [
"Djibouti",
"Eritrea",
"Ethiopia",
"Kenya",
"Rwanda",
"Somalia",
"South Sudan",
"Sudan",
"Tanzania",
"Uganda"]

eastern_df = wdi_melt[wdi_melt['Country Name'].isin(eastern_countries)]
print(eastern_df.shape)
eastern_df.head()

(930510, 6)


,Country Name,Country Code,Indicator Name,Indicator Code,Years,Score
152131,Djibouti,DJI,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,1960,NaN
152132,Djibouti,DJI,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,1960,NaN
152133,Djibouti,DJI,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,1960,NaN
152134,Djibouti,DJI,Access to electricity (% of population),EG.ELC.ACCS.ZS,1960,NaN
152135,Djibouti,DJI,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,1960,NaN


In [7]:
expenditure = [i for i in eastern_df['Indicator Name'] if 'expenditure' in i]
expenditure_df = eastern_df[eastern_df['Indicator Name'].isin(expenditure)]
expenditure_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53550 entries, 152167 to 24730622
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    53550 non-null  object 
 1   Country Code    53550 non-null  object 
 2   Indicator Name  53550 non-null  object 
 3   Indicator Code  53550 non-null  object 
 4   Years           53550 non-null  object 
 5   Score           17030 non-null  float64
dtypes: float64(1), object(5)
memory usage: 2.9+ MB


In [8]:
expenditure_df = expenditure_df[(expenditure_df['Years'] >= '2010') & (expenditure_df['Years'] <= '2021')]

In [9]:
merged_df = pd.merge(expenditure_df, 
                     series_df,
                     left_on="Indicator Code",
                     right_on="Series Code",
                     how='left')
merged_df.head(2)

,Country Name,Country Code,Indicator Name_x,Indicator Code,Years,Score,Series Code,Topic,Indicator Name_y,Short definition,...,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
0,Djibouti,DJI,Adjusted savings: education expenditure (% of ...,NY.ADJ.AEDU.GN.ZS,2010,7.802883e+00,NY.ADJ.AEDU.GN.ZS,Economic Policy & Debt: National accounts: Adj...,Adjusted savings: education expenditure (% of ...,NaN,...,NaN,NaN,World Bank staff estimates using data from the...,NaN,NaN,NaN,NaN,NaN,CC BY-4.0,NaN
1,Djibouti,DJI,Adjusted savings: education expenditure (curre...,NY.ADJ.AEDU.CD,2010,8.941872e+07,NY.ADJ.AEDU.CD,Economic Policy & Debt: National accounts: Adj...,Adjusted savings: education expenditure (curre...,NaN,...,NaN,NaN,World Bank staff estimates using data from the...,NaN,NaN,NaN,NaN,NaN,CC BY-4.0,NaN


In [10]:
neccessary_columns = ['Country Name', 'Country Code', 'Indicator Name_x', 'Indicator Code', 'Topic', 'Years', 'Score', 'Aggregation method']

merged_df = merged_df[neccessary_columns]
merged_df.head()

,Country Name,Country Code,Indicator Name_x,Indicator Code,Topic,Years,Score,Aggregation method
0,Djibouti,DJI,Adjusted savings: education expenditure (% of ...,NY.ADJ.AEDU.GN.ZS,Economic Policy & Debt: National accounts: Adj...,2010,7.802883e+00,Weighted average
1,Djibouti,DJI,Adjusted savings: education expenditure (curre...,NY.ADJ.AEDU.CD,Economic Policy & Debt: National accounts: Adj...,2010,8.941872e+07,NaN
2,Djibouti,DJI,"Current education expenditure, primary (% of t...",SE.XPD.CPRM.ZS,Education: Inputs,2010,NaN,Median
3,Djibouti,DJI,"Current education expenditure, secondary (% of...",SE.XPD.CSEC.ZS,Education: Inputs,2010,NaN,Median
4,Djibouti,DJI,"Current education expenditure, tertiary (% of ...",SE.XPD.CTER.ZS,Education: Inputs,2010,NaN,Median


In [11]:
health = [topic for topic in merged_df['Topic'] if topic.startswith("Health: Health systems")]
health_df = merged_df[merged_df['Topic'].isin(health)]
health_df.head()

,Country Name,Country Code,Indicator Name_x,Indicator Code,Topic,Years,Score,Aggregation method
6,Djibouti,DJI,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,2010,3.061504,Weighted average
7,Djibouti,DJI,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,Health: Health systems,2010,51.371128,Weighted average
8,Djibouti,DJI,"Current health expenditure per capita, PPP (cu...",SH.XPD.CHEX.PP.CD,Health: Health systems,2010,100.435942,Weighted average
11,Djibouti,DJI,Domestic general government health expenditure...,SH.XPD.GHED.CH.ZS,Health: Health systems,2010,60.679157,Weighted average
12,Djibouti,DJI,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,Health: Health systems,2010,1.857695,Weighted average


In [12]:
health_df['Years'] = pd.to_datetime(health_df['Years'])
health_df.set_index('Years', inplace=True)
health_df.head()

,Country Name,Country Code,Indicator Name_x,Indicator Code,Topic,Score,Aggregation method
Years,,,,,,,
2010-01-01,Djibouti,DJI,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,3.061504,Weighted average
2010-01-01,Djibouti,DJI,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,Health: Health systems,51.371128,Weighted average
2010-01-01,Djibouti,DJI,"Current health expenditure per capita, PPP (cu...",SH.XPD.CHEX.PP.CD,Health: Health systems,100.435942,Weighted average
2010-01-01,Djibouti,DJI,Domestic general government health expenditure...,SH.XPD.GHED.CH.ZS,Health: Health systems,60.679157,Weighted average
2010-01-01,Djibouti,DJI,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,Health: Health systems,1.857695,Weighted average


In [13]:
# Renaming columns
health_df.rename(columns={
    'Country Name': 'Country',
    'Indicator Name_x': 'Indicator Name'
}, inplace=True)
health_df.head()

,Country,Country Code,Indicator Name,Indicator Code,Topic,Score,Aggregation method
Years,,,,,,,
2010-01-01,Djibouti,DJI,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,3.061504,Weighted average
2010-01-01,Djibouti,DJI,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,Health: Health systems,51.371128,Weighted average
2010-01-01,Djibouti,DJI,"Current health expenditure per capita, PPP (cu...",SH.XPD.CHEX.PP.CD,Health: Health systems,100.435942,Weighted average
2010-01-01,Djibouti,DJI,Domestic general government health expenditure...,SH.XPD.GHED.CH.ZS,Health: Health systems,60.679157,Weighted average
2010-01-01,Djibouti,DJI,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,Health: Health systems,1.857695,Weighted average


In [30]:
def create_dfs(df):
    indicators = [i for i in df['Indicator Name'].unique()]
    dfs = {}
    for i in indicators:
        dfs[i] = df[df['Indicator Name'] == i]
    return dfs

def create_country_dfs(df):
    countries = [country for country in df['Country'].unique()]
    indicator_dfs = create_dfs(df) 
    country_dfs = {}
    
    for c in countries:
        country_dfs[c] = {} 
        for indicator, indicator_df in indicator_dfs.items():
            country_df = indicator_df[indicator_df['Country'] == c]
            country_df['Percentage Change'] = country_df['Score'].pct_change() * 100
            country_dfs[c][indicator] = country_df
            
    return country_dfs

In [31]:
countries = create_country_dfs(health_df)
countries['Kenya']['Current health expenditure (% of GDP)']

,Country,Country Code,Indicator Name,Indicator Code,Topic,Score,Aggregation method,Percentage Change
Years,,,,,,,,
2010-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,5.388498,Weighted average,NaN
2011-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,5.202128,Weighted average,-3.458661
2012-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,5.018058,Weighted average,-3.538362
2013-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,4.931004,Weighted average,-1.734819
2014-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,4.912673,Weighted average,-0.371741
2015-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,4.764396,Weighted average,-3.018270
2016-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,4.754291,Weighted average,-0.212087
2017-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,3.985975,Weighted average,-16.160475
2018-01-01,Kenya,KEN,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,Health: Health systems,4.099034,Weighted average,2.836415
